In [2]:
!pip install mysql-connector-python
!pip install pandas
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [3]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [4]:
connection = create_server_connection("localhost", "root", "")

MySQL Database connection successful


In [5]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [6]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [8]:
conn1 = create_db_connection("localhost", "root", "", "schedmaster")

MySQL Database connection successful


In [9]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [10]:
q1 = """
SELECT *
FROM students;
"""
results = read_query(conn1, q1)
for result in results:
  print(result)

('aaa111', 'John', 'Doe', 'CPSC', 120)


In [12]:
print(results[0][0])

aaa111


In [15]:
class Student(object):
    def __init__(self, id, firstName, lastName, major, requiredHours):
        self.id = id
        self.firstName = firstName
        self.lastName = lastName
        self.major = major
        self.requiredHours = requiredHours

In [17]:
jimmy = Student('abc123', 'Jimmy', 'Mife', 'CPSC', 120)

In [23]:
q2 = """
SELECT c.meeting_time_start, c.meeting_time_end, c.days
FROM sections c;"""

In [29]:
courses = read_query(conn1, q2)

In [30]:
import datetime
datetime.timedelta(seconds=36000) < datetime.timedelta(seconds=48000)

True

In [31]:
class Course(object):
    def __init__(self, startTime, endTime, days):
        self.days = days
        self.startTime = startTime
        self.endTime = endTime
        # Note that the final version of this class will have other fields too.
    def conflictsWith(self, course):
        timeOverlap = False
        dayOverlap = False
        if ((self.startTime < course.startTime and self.endTime > course.startTime) 
            or (self.startTime > course.startTime and self.startTime < course.endTime) 
            or self.startTime == course.startTime or self.endTime == course.endTime):
            timeOverlap = True
        for day in self.days:
            if (day in course.days): dayOverlap = True
        if (timeOverlap and dayOverlap): return True
        else: return False

In [41]:
for c in range(10):
    print(courses[c])

(None, None, 'N/A')
(datetime.timedelta(seconds=36000), datetime.timedelta(seconds=42600), 'M')
(datetime.timedelta(seconds=56400), datetime.timedelta(seconds=63000), 'M')
(datetime.timedelta(seconds=31800), datetime.timedelta(seconds=38400), 'T')
(datetime.timedelta(seconds=39000), datetime.timedelta(seconds=45600), 'T')
(datetime.timedelta(seconds=49200), datetime.timedelta(seconds=55800), 'T')
(datetime.timedelta(seconds=39600), datetime.timedelta(seconds=42600), 'MWF')
(datetime.timedelta(seconds=46800), datetime.timedelta(seconds=53400), 'R')
(datetime.timedelta(seconds=33900), datetime.timedelta(seconds=38400), 'TR')
(datetime.timedelta(seconds=32400), datetime.timedelta(seconds=35400), 'MWF')


In [65]:
allCourses = []
for c in courses:
    if c[0] != None:
        newCourse = Course(c[0], c[1], c[2])
        allCourses.append(newCourse)

In [66]:
allCourses[1]

In [67]:
allCourses[1].startTime

datetime.timedelta(seconds=56400)

In [69]:
allCourses[0].days

'M'

In [70]:
allCourses[0].conflictsWith(allCourses[1])

False

In [71]:
allCourses[0].conflictsWith(allCourses[5])

True

In [72]:
def nonconflicting(studCourses, allCourses):
    conf = []
    nonconf = []
    for course1 in studCourses:
        for course2 in allCourses:
            if (course1.conflictsWith(course2)):
                if course2 not in conf:
                    conf.append(course2)
    for course in allCourses:
        if course not in conf:
            nonconf.append(course)
    return nonconf

In [73]:
courseA = allCourses[0]
courseB = allCourses[2]
studCourses = [courseA, courseB]
testCourses = allCourses[:10]
nonconf = nonconflicting(studCourses, testCourses)

In [78]:
for c in nonconf:
    print(str(c.startTime) + ', ' + str(c.endTime) + ', ' + c.days)

15:40:00, 17:30:00, M
10:50:00, 12:40:00, T
13:40:00, 15:30:00, T
13:00:00, 14:50:00, R
9:00:00, 9:50:00, MWF
18:00:00, 20:30:00, M


In [79]:
print(str(courseA.startTime) + ', ' + str(courseA.endTime) + ', ' + courseA.days)
print(str(courseB.startTime) + ', ' + str(courseB.endTime) + ', ' + courseB.days)

10:00:00, 11:50:00, M
8:50:00, 10:40:00, T
